In [1575]:
from graphviz import Digraph
import re
import json

In [1576]:
lowercase_alphabets = [chr(i) for i in range(ord('a'), ord('z') + 1)]
uppercase_alphabets = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
digits = [str(i) for i in range(10)]
# all lists
alphabets_and_digits = lowercase_alphabets + uppercase_alphabets + digits
#print("Alphabets and Digits:", alphabets_and_digits)

In [1577]:
def insert(str1, str2, idx):
    return str1[:idx] + str2 + str1[idx:]

In [1578]:
def is_valid_regex(regex):
    try:
        re.compile(regex)
        return True
    except re.error:
        return False

In [1579]:
def range_handling(infix):
    range_handling_infix = []
    flage = False
    i = 0
    while i < len(infix):
        char = infix[i]
        range_handling_infix.append(char)
        
        if char == '[':
            flage = True
        elif char == ']':
            flage = False
        elif flage and char in alphabets_and_digits and i + 1 < len(infix) and infix[i + 1] in alphabets_and_digits:
            range_handling_infix.append('|')
        
        i += 1

    result = ''.join(range_handling_infix)
    return result

In [1580]:
def parse(infix):
    parsed_regex = []
    operators = ['|', '+', '*', '?', '-', '(', ')', '[', ']', '.']
    for i in range(len(infix) - 1):
        current = infix[i]
        parsed_regex.append(current)
        next_char = infix[i + 1]
        # conditions when to concatenate 
        if current not in operators and (next_char not in operators or next_char == '(' or next_char == '['):
            parsed_regex.append('.')
        if (current == ')' or current == ']') and (next_char not in operators or next_char == '(' or next_char == '['):
            parsed_regex.append('.')
        if (current in ['*', '+', '?']) and (next_char == '(' or next_char == '[' or next_char not in operators):
            parsed_regex.append('.')
    # the last character of regex
    parsed_regex.append(infix[-1])

    return ''.join(parsed_regex)

In [1581]:
def infix_to_postfix(infix):
    precedence_order = {
        '|': 1, 
        '.': 2, 
        '?': 3, 
        '+': 4, 
        '*': 5 
    }
    stack = []
    postfix = ''
    i = 0
    while infix and i < len(infix):
        char = infix[i]
        if char.isalnum():
            postfix += char
        elif char == "(" or char == "[":
            stack.append(char) 
        elif char == ")":
            while stack and stack[-1] != "(":
                postfix += stack.pop()  
            stack.pop() 
        elif char == "]":
            while stack and stack[-1] != "[":
                postfix += stack.pop()
            stack.pop()
        elif char in precedence_order:
            while stack and precedence_order[char] <= precedence_order.get(stack[-1], 0):  
                postfix += stack.pop()                
            stack.append(char)                                                
        elif char == "-":
            range_start = postfix[-1]
            range_end = infix[i+1]
            range = []
            for c in alphabets_and_digits:
                if(alphabets_and_digits.index(c) > alphabets_and_digits.index(range_start) and alphabets_and_digits.index(c) < alphabets_and_digits.index(range_end)):
                    range.append('|')
                    range.append(c)
            range.append('|')
            infix = insert(infix, ''.join(range), i+1)
           
        i+=1

    while stack:
        postfix += stack.pop()

    return postfix

In [1582]:
class Edge:
    def __init__(self, label=None, destination=None):
        self.label = label if label is not None else ""
        self.destination = destination if destination is not None else ""
        
    def __str__(self):
        return f"Edge with label '{self.label}' going to {self.destination}"

class Node:
  def __init__(self):
    self.label=None
    self.Out_Edges=[]

  def __repr__(self) -> str:
     return self.label
  def __eq__(self, other):
    return other.label == self.label
  def __ne__(self, other):
    return other.label != self.label
  def __hash__(self):
     return hash(self.label)

class nfa:
    def __init__(self, initial, accept, inner):
        self.start = initial
        self.accept = accept
        self.inner_states = inner
    def __repr__(self) ->str:
      return self.start.label

In [1583]:
def visualize_NFA(result_NFA):
  gra = Digraph(graph_attr={'rankdir':'LR'})
  #construct nodes first
  for stat in result_NFA.inner_states:
      if(stat.label == result_NFA.start.label):
        gra.node("", _attributes={'shape' : 'none'})
        gra.edge("", stat.label)
      if(stat.label == result_NFA.accept.label):
        gra.node(stat.label, shape="doublecircle")
      else:
        gra.node(stat.label, shape="circle")
  #for each node, construct edges
  for stat in result_NFA.inner_states:
      for edg in stat.Out_Edges:
          gra.edge(stat.label, edg.destination.label, label=edg.label)
  gra.format = 'pdf'
  gra.render('NFA', view = True)
  return gra.source

In [1584]:
def PostFix_To_NFA(postfix):
    stack = []
    id = 0
    for c in postfix:
        if c in alphabets_and_digits:
            start = Node()
            accept = Node()
            start.label = "S" + str(id)
            accept.label = "S" + str(id + 1)

            nEdge = Edge()
            nEdge.label = c
            nEdge.destination = accept
            start.Out_Edges.append(nEdge)
            
            result_nfa = nfa(start, accept, [start, accept])
            stack.append(result_nfa)
            id += 2

        elif c == '*':
            nfa1 = stack.pop()

            newStart = Node()
            newStart.label = "S" + str(id)
            newEnd = Node()
            newEnd.label = "S" + str(id + 1)

            nEdge1 = Edge()
            nEdge1.label = 'epsilon'
            nEdge1.destination = newStart
            nfa1.accept.Out_Edges.append(nEdge1)

            nEdge2 = Edge()
            nEdge2.label = 'epsilon'
            nEdge2.destination = nfa1.start
            newStart.Out_Edges.append(nEdge2)

            nEdge3 = Edge()
            nEdge3.label = 'epsilon'
            nEdge3.destination = newEnd
            nfa1.accept.Out_Edges.append(nEdge3)

            nEdge4 = Edge()
            nEdge4.label = 'epsilon'
            nEdge4.destination = newEnd
            newStart.Out_Edges.append(nEdge4)

            result = nfa(newStart, newEnd, [newStart, newEnd] + nfa1.inner_states)
            stack.append(result)
            id += 2

        elif c == '+':
            nfa1 = stack.pop()

            newStart = Node()
            newStart.label = "S" + str(id)
            newEnd = Node()
            newEnd.label = "S" + str(id + 1)

            nEdge1 = Edge()
            nEdge1.label = 'epsilon'
            nEdge1.destination = newStart
            nfa1.accept.Out_Edges.append(nEdge1)

            nEdge2 = Edge()
            nEdge2.label = 'epsilon'
            nEdge2.destination = nfa1.start
            newStart.Out_Edges.append(nEdge2)

            nEdge3 = Edge()
            nEdge3.label = 'epsilon'
            nEdge3.destination = newEnd
            nfa1.accept.Out_Edges.append(nEdge3)

            result = nfa(newStart, newEnd, [newStart, newEnd] + nfa1.inner_states)
            stack.append(result)
            id += 2

        elif c == '?':
            nfa1 = stack.pop()

            newStart = Node()
            newStart.label = "S" + str(id)
            newEnd = Node()
            newEnd.label = "S" + str(id + 1)

            nEdge1 = Edge()
            nEdge1.label = 'epsilon'
            nEdge1.destination = nfa1.start
            newStart.Out_Edges.append(nEdge1)

            nEdge2 = Edge()
            nEdge2.label = 'epsilon'
            nEdge2.destination = newEnd
            nfa1.accept.Out_Edges.append(nEdge2)

            nEdge2 = Edge()
            nEdge2.label = 'epsilon'
            nEdge2.destination = newEnd
            newStart.Out_Edges.append(nEdge2)

            result = nfa(newStart, newEnd, [newStart, newEnd] + nfa1.inner_states)
            stack.append(result)
            id += 2

        elif c == '.':
            nfa2 = stack.pop()
            nfa1 = stack.pop()

            nEdge = Edge()
            nEdge.label = 'epsilon'
            nEdge.destination = nfa2.start
            nfa1.accept.Out_Edges.append(nEdge)

            resultnfa = nfa(nfa1.start,  nfa2.accept, nfa1.inner_states + nfa2.inner_states)
            stack.append(resultnfa)

        elif c == '|':
            nfa1 = stack.pop()
            nfa2 = stack.pop()
            newStart = Node()
            newStart.label= "S"+str(id)
            newEnd = Node()
            newEnd.label = "S"+str(id+1)

            nEdge1 = Edge()
            nEdge1.label = 'epsilon'
            nEdge1.destination = nfa1.start

            nEdge2 = Edge()
            nEdge2.label = 'epsilon'
            nEdge2.destination = nfa2.start

            newStart.Out_Edges.append(nEdge1)
            newStart.Out_Edges.append(nEdge2)

            nEdge3 = Edge()
            nEdge3.label = 'epsilon'
            nEdge3.destination = newEnd
            nfa1.accept.Out_Edges.append(nEdge3)

            nEdge4 = Edge()
            nEdge4.label = 'epsilon'
            nEdge4.destination = newEnd
            nfa2.accept.Out_Edges.append(nEdge4)

            result = nfa(newStart, newEnd, [newStart,newEnd]+ nfa1.inner_states + nfa2.inner_states)
            stack.append(result)
            id += 2

    result = stack.pop()
    return result, visualize_NFA(result)


In [1585]:
def construct_NFA_json (nnfa:nfa):
  outputJson = dict()
  outputJson["startingState"] = nnfa.start.label
  for stat in nnfa.inner_states:
    stateDict = dict()
    if stat == nnfa.accept:
      stateDict["isTerminatingState"] = True
    else:
      stateDict["isTerminatingState"] = False
    for edg in stat.Out_Edges:
      if(edg.label in stateDict.keys()):
        stateDict[edg.label] = [] + [stateDict[edg.label]] + [edg.destination.label]
      else:
        stateDict[edg.label] = edg.destination.label
    outputJson[stat.label] = stateDict
  nfaOutFile = open('NFA.json', 'w')
  JsonObject = json.dump(outputJson, nfaOutFile,indent=6, ensure_ascii=False)
  nfaOutFile.close()
  return JsonObject

In [1587]:
inputRegex = "(e)"
if not is_valid_regex(inputRegex):
        print("error, invalid regex")
else:
        inputRegex_with_handled_classes = range_handling(inputRegex)
        inputRegex_Preprocessed = parse(inputRegex_with_handled_classes)
        postfix = infix_to_postfix(inputRegex_Preprocessed)

        print(PostFix_To_NFA(postfix))
        nfa, _ = PostFix_To_NFA(postfix)
        construct_NFA_json(nfa)

(S0, 'digraph {\n\tgraph [rankdir=LR]\n\t"" [shape=none]\n\t"" -> S0\n\tS0 [shape=circle]\n\tS1 [shape=doublecircle]\n\tS0 -> S1 [label=e]\n}\n')
